<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-09 15:12:40
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 25.67 K (0.19%)
Current PnL: -12.96 L (-9.03%)
CY Booked + Current PnL: -6.00 L (-4.18%)
-------------------
Total profit:  4.95 L
Total loss:  -17.91 L
-------------------
Total Booked + Current PnL: 20.98 L (18.0%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.48 L (52.12%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.97%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-218.54,H-SC,98.96,85498.0,-9818.0,9858.0,4.07,-10.30,11.53,0.04,157.0,-1.00,0.62,68.63,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-17.63,H-LC,90.62,112170.0,8604.0,12103.0,-1.15,8.31,10.79,20.00,32.0,0.71,0.82,23.55,X40N,BTT,HEALTHCARE
71,SBIN,760.30,863.00,-11.51,M-LC,33.33,214915.0,13436.0,13776.0,-0.20,6.67,6.41,13.51,77.0,0.98,1.57,20.11,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-21.55,H-LC,71.88,107864.0,3552.0,13979.0,-1.67,3.41,12.96,16.81,27.0,0.25,0.79,20.00,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,26.04,128277.0,-14202.0,14829.0,0.81,-9.97,11.56,0.44,247.0,-0.96,0.94,51.40,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-32.52,M-MC,8.33,178331.0,-1477.0,113882.0,-0.86,-0.82,63.86,62.52,173.0,-0.01,1.30,32.99,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,39.06,151672.0,759.0,59698.0,0.77,0.50,39.36,40.06,103.0,0.01,1.11,24.06,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,MEDANTA,1064.65,1486.0,16.12,X-SC,88.54,247503.0,42026.0,39303.0,5.28,20.45,15.88,39.58,65.0,1.07,1.81,32.63,XY24,NTT,HEALTHCARE
92,VIPIND,488.80,489.0,-218.54,H-SC,98.96,85498.0,-9818.0,9858.0,4.07,-10.30,11.53,0.04,157.0,-1.00,0.62,68.63,OX40N,NTT,MISC
55,LALPATHLAB,2804.19,3545.0,-29.11,H-SC,69.79,163207.0,8977.0,31760.0,3.03,5.82,19.46,26.42,127.0,0.28,1.19,28.70,X40N,NTT,HEALTHCARE
69,SAMMAANCAP,170.35,326.0,-167.05,M-SC,55.21,77016.0,-25194.0,118582.0,2.62,-24.65,153.97,91.37,212.0,-0.21,0.56,25.02,XY25,NTT,FINANCE
32,ENRIN,1377.95,3124.0,NaN,NaN,95.83,78100.0,43651.0,0.0,2.53,126.71,0.00,126.71,274.0,inf,0.57,19.43,AR,ATH,ELECTRICAL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.00,-6.01,M-LC,6.25,164069.0,23229.0,21263.0,-3.96,16.49,12.96,31.59,88.0,1.09,1.20,42.52,XY24,NTT,BANKS
28,DIGITIDE,188.38,237.72,NaN,NaN,18.75,49069.0,7814.0,2993.0,-3.81,18.94,6.10,26.19,272.0,2.61,0.36,15.52,XY24,ATH,IT
41,HINDZINC,514.80,730.22,13.20,H-LC,1.04,85080.0,-17880.0,60960.0,-2.48,-17.37,71.65,41.85,24.0,-0.29,0.62,11.98,X5K,ATH,METALS
0,5PAISA,564.75,565.00,148.80,H-SC,17.71,134469.0,-52463.0,52550.0,-2.40,-28.07,39.08,0.04,149.0,-1.00,0.98,28.64,OX40N,NTT,FINANCE
74,SIEMENS,4406.90,7969.85,-4.80,X-LC,13.54,80505.0,-29667.0,118745.0,-2.32,-26.93,147.50,80.85,11.0,-0.25,0.59,18.57,AR,ATH,ELECTRICAL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,26.04,128277.0,-14202.0,14829.0,0.81,-9.97,11.56,0.44,247.0,-0.96,0.94,51.40,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-218.54,H-SC,98.96,85498.0,-9818.0,9858.0,4.07,-10.30,11.53,0.04,157.0,-1.00,0.62,68.63,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-29.18,M-SC,25.00,151069.0,-20072.0,20017.0,-0.07,-11.73,13.25,-0.03,191.0,-1.00,1.10,8.58,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148924.0,-25635.0,25674.0,0.06,-14.69,17.24,0.02,133.0,-1.00,1.09,29.14,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.98,X-MC,9.38,86128.0,-16524.0,16519.0,-0.32,-16.10,19.18,-0.00,59.0,-1.00,0.63,30.92,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.02,X-LC,34.38,264046.0,-31888.0,125792.0,-0.62,-10.78,47.64,31.73,1.0,-0.25,1.93,5.69,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,76.04,292504.0,30903.0,114720.0,-0.28,11.81,39.22,55.67,2.0,0.27,2.13,18.64,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.30,X-LC,61.98,271354.0,1007.0,33621.0,1.26,0.37,12.39,12.81,5.0,0.03,1.98,13.49,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.95,X-LC,68.23,270549.0,41316.0,25973.0,0.73,18.02,9.60,29.35,10.0,1.59,1.97,28.55,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-4.80,X-LC,13.54,80505.0,-29667.0,118745.0,-2.32,-26.93,147.50,80.85,11.0,-0.25,0.59,18.57,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3124.0,NaN,NaN,95.83,78100.0,43651.0,0.0,2.53,126.71,0.00,126.71,274.0,inf,0.57,19.43,AR,ATH,ELECTRICAL
59,MEDANTA,1064.65,1486.0,16.12,X-SC,88.54,247503.0,42026.0,39303.0,5.28,20.45,15.88,39.58,65.0,1.07,1.81,32.63,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.93,M-MC,43.75,230524.0,5562.0,162081.0,-0.87,2.47,70.31,74.53,189.0,0.03,1.68,33.84,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,27.10,H-SC,51.04,130145.0,4848.0,54270.0,1.82,3.87,41.70,47.18,143.0,0.09,0.95,62.82,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,30.21,117309.0,15305.0,29714.0,1.65,15.00,25.33,44.14,126.0,0.52,0.86,33.68,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-2.86,M-LC,85.42,166098.0,15153.0,94294.0,-0.65,10.04,56.77,72.51,71.0,0.16,1.21,14.41,XR,NTT,IT
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181000.0,16330.0,52762.0,-0.01,9.92,29.15,41.95,124.0,0.31,1.32,18.12,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3124.00,NaN,NaN,95.83,78100.0,43651.0,0.0,2.53,126.71,0.00,126.71,274.0,inf,0.57,19.43,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-39.82,M-SC,39.06,198376.0,26755.0,135312.0,1.24,15.59,68.21,94.43,201.0,0.20,1.45,46.41,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,85.42,166098.0,15153.0,94294.0,-0.65,10.04,56.77,72.51,71.0,0.16,1.21,14.41,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-47.33,H-SC,80.21,137090.0,13754.0,123216.0,1.29,11.15,89.88,111.05,123.0,0.11,1.00,34.32,AR,ATH,MISC
54,KPIGREEN,497.21,731.79,27.10,H-SC,51.04,130145.0,4848.0,54270.0,1.82,3.87,41.70,47.18,143.0,0.09,0.95,62.82,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.02,NaN,NaN,58.33,18838.0,-30755.0,0.0,1.53,-62.01,0.00,-62.01,270.0,-inf,0.14,26.33,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,73.96,65837.0,-47712.0,87761.0,2.21,-42.02,133.30,35.27,265.0,-0.54,0.48,61.14,XR,NTT,HOTELS
32,ENRIN,1377.95,3124.00,NaN,NaN,95.83,78100.0,43651.0,0.0,2.53,126.71,0.00,126.71,274.0,inf,0.57,19.43,AR,ATH,ELECTRICAL
92,VIPIND,488.80,489.00,-218.54,H-SC,98.96,85498.0,-9818.0,9858.0,4.07,-10.30,11.53,0.04,157.0,-1.00,0.62,68.63,OX40N,NTT,MISC
58,MASFIN,326.60,399.50,-17.80,H-SC,66.67,93480.0,-4500.0,26371.0,0.00,-4.59,28.21,22.32,145.0,-0.17,0.68,35.58,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-16.18,L-SC,100.00,168878.0,27036.0,29537.0,-0.40,19.06,17.49,39.88,262.0,0.92,1.23,71.14,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-218.54,H-SC,98.96,85498.0,-9818.0,9858.0,4.07,-10.30,11.53,0.04,157.0,-1.00,0.62,68.63,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.43,X-SC,97.92,99979.0,-984.0,95510.0,-1.31,-0.97,95.53,93.63,67.0,-0.01,0.73,48.92,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181000.0,16330.0,52762.0,-0.01,9.92,29.15,41.95,124.0,0.31,1.32,18.12,X40N,ATH,AC
32,ENRIN,1377.95,3124.00,NaN,NaN,95.83,78100.0,43651.0,0.0,2.53,126.71,0.00,126.71,274.0,inf,0.57,19.43,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.02
1,25,42.03
2,50,70.70


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.55
LC    33.86
MC    21.57
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.00
X40      12.95
XY25     12.09
X40N     11.99
XR        9.98
AR        8.19
OX40N     8.16
MH        1.67
X5K       1.41
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.52
M-SC    15.87
H-LC    14.10
X-LC    12.89
H-MC     9.58
X-MC     7.60
M-LC     5.82
X-SC     4.54
M-MC     4.04
L-SC     2.62
L-LC     1.05
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.28,0.06,33.34
IT,12.06,-10.05,64.14
FINANCE,8.51,-11.71,52.25
BANKS,7.65,-8.86,53.28
MISC,5.87,-19.59,75.67
PAINTS,5.70,-11.73,33.76
ELECTRICAL,5.46,1.48,45.03
HEALTHCARE,4.75,0.95,27.98
AUTO,4.30,-14.79,65.26


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2643432.0,26
XR,1089682.0,13
AR,1030100.0,10
X40,679592.0,10
X40N,522831.0,11
XY25,462534.0,8
OX40N,311160.0,11
SR,178709.0,2
MH,86579.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1973661.0,20
M-SC,1768966.0,21
H-LC,610459.0,12
X-LC,542411.0,8
H-MC,522886.0,8
M-MC,415882.0,3
X-MC,411268.0,6
X-SC,296423.0,5
M-LC,264057.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      887742.0      6
M-SC       XY24      805873.0      7
H-SC       AR        526147.0      3
M-SC       XR        338204.0      4
M-MC       XY24      302000.0      2
H-MC       XY24      297383.0      3
X-LC       X40       276344.0      3
H-SC       XR        216589.0      3
           X40N      200831.0      4
H-LC       X40N      193051.0      4
M-SC       AR        187161.0      2
           OX40N     167007.0      5
M-LC       XY24      155987.0      2
H-LC       X40       154877.0      3
X-MC       X40       154593.0      2
L-SC       XR        149963.0      2
X-SC       XR        122058.0      1
X-LC       AR        118745.0      1
M-SC       XY25      118582.0      1
M-MC       XR        113882.0      1
X-MC       XY25      103551.0      1
X-LC       XY25       97782.0      3
X-SC       SR         95510.0      1
M-LC       XR         94294.0      1
H-SC       OX40N      88082.0      3
M-SC       SR         83199.0      1
X-MC       XY24       83036.0      1
H-MC       XY25       82818.0      1
H-LC       AR         79436.0      1
           X5K        74939.0      2
M-SC       X40        68940.0      1
H-LC       X200       68578.0      1
H-MC       AR         59698.0      1
L-SC       AR         58913.0      1
L-MC       XR         54692.0      1
H-SC       MH         54270.0      1
X-MC       X40N       53569.0      1
X-LC       X40N       49540.0      1
L-LC       XY25       46025.0      1
H-LC       XY24       39578.0      1
X-SC       OX40N      39552.0      2
           XY24       39303.0      1
H-MC       MH         32309.0      1
L-SC       XY24       29537.0      1
H-MC       X40N       25840.0      1
           X40        24838.0      1
X-MC       OX40N      16519.0      1
M-LC       XY25       13776.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
